In [4]:
from functools import reduce
import operator

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [10]:
df = spark.read.option('header','true').csv(r'C:\\users\102222\Downloads\hmm.csv')

In [23]:
df.show(5)

+-------------------+--------+--------+--------+--------+------+
|          timestamp|    open|    high|     low|   close|volume|
+-------------------+--------+--------+--------+--------+------+
|2022-06-03 20:00:00|141.4700|141.4700|141.3500|141.3500|   904|
|2022-06-03 19:55:00|141.5000|141.5000|141.5000|141.5000|   509|
|2022-06-03 19:15:00|141.6500|141.6500|141.4700|141.4700|  1000|
|2022-06-03 19:00:00|141.5900|141.5900|141.5900|141.5900|   200|
|2022-06-03 18:35:00|141.7000|141.7000|141.7000|141.7000|   100|
+-------------------+--------+--------+--------+--------+------+
only showing top 5 rows



In [21]:
df.filter((col('open')>140)&(col('volume')<300)).count()

2

In [22]:
df.filter((col('open')>140)|(col('volume')<300)).count()

64

In [24]:
data = {
    'filter' : [
        {
            'name' : 'open',
            'comparisonType' : '>',
            'value' : 140
        },
        {
            'name' : 'volume',
            'comparisonType' : '<',
            'value' : 300
        }
    ],
    'matchAll' : True
}

In [47]:
values = {
    '>' : operator.gt,
    '<' : operator.lt
}

def getFunc(func,column,value):
    return func(column,value)

In [48]:
filters = []
for filter_col in data['filter']:
    filters.append(getFunc(values.get(filter_col.get('comparisonType')),col(filter_col.get('name')),filter_col.get('value')))

In [49]:
list(
map(
lambda col_name : getFunc(values.get(col_name.get('comparisonType')),col(col_name.get('name')),col_name.get('value')),data['filter']
)
)

[Column<'(open > 140)'>, Column<'(volume < 300)'>]

In [51]:
if data['matchAll']:
    filt = reduce(operator.and_,filters)

In [54]:
df.filter(filt).show()

+-------------------+--------+--------+--------+--------+------+
|          timestamp|    open|    high|     low|   close|volume|
+-------------------+--------+--------+--------+--------+------+
|2022-06-03 19:00:00|141.5900|141.5900|141.5900|141.5900|   200|
|2022-06-03 18:35:00|141.7000|141.7000|141.7000|141.7000|   100|
+-------------------+--------+--------+--------+--------+------+



In [58]:
from typing import Dict

operator_dict = {
    '>' : operator.gt,
    '<' : operator.lt
}


def MutliFilters(data: Dict):
    def __operatorFunc(op, column, value):
        return op(column, value)

    filters = list(
        map(
            lambda col_name: __operatorFunc(
                operator_dict.get(col_name.get("comparisonType")),
                col(col_name.get("name")),
                col_name.get("value"),
            ),
            data["filter"],
        )
    )
    if data.get("matchAll"):
        return reduce(operator.and_, filters)
    else:
        return reduce(operator.or_, filters)

In [60]:
data_and = {
    'filter' : [
        {
            'name' : 'open',
            'comparisonType' : '>',
            'value' : 140
        },
        {
            'name' : 'volume',
            'comparisonType' : '<',
            'value' : 300
        }
    ],
    'matchAll' : True
}

In [70]:
df.filter(MutliFilters(data_and)).count()

2

In [68]:
data_or = {
    'filter' : [
        {
            'name' : 'open',
            'comparisonType' : '>',
            'value' : 140
        },
        {
            'name' : 'volume',
            'comparisonType' : '<',
            'value' : 300
        }
    ],
    'matchAll' : False
}

In [71]:
df.filter(MutliFilters(data_or)).count()

64